In [2]:
%matplotlib inline
#
from lrgsglib.shared import *
from lrgsglib.config.utils import *
from lrgsglib.core import gaussian_filter, Lattice2D, \
    read_files_to_2d_array, move_to_rootf, eigV_for_lattice2D,\
    create_symmetric_log_bins, bin_eigenvalues
from matplotlib.ticker import SymmetricalLogLocator
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.colorbar import ColorbarBase
#
move_to_rootf()
plt.style.use("ipynb/nb_plotsheet.mplstyle")
#
subDatPth = os.path.join('plot', 'paper', 'suppinf', '')
sgPath = os.path.join(subDatPth, 'spectra_test_1', '')
sgPath2 = os.path.join(subDatPth, 'spectra_test_2', '')
datPth = os.path.join('data', subDatPth)
subPthStr = ['spectra_test_1', 'spectra_test_2', 'res_limit']
datPthDict = {s: os.path.join(datPth, s, '') for s in subPthStr}
for path in datPthDict.values(): os.makedirs(path, exist_ok=True)
minExpRL = np.rint(np.log10(np.finfo(np.float64).eps)).astype(int)

cwd: /home/opisthofulax/PhD/projects/complex-networks/lrgsglib


## Precision issues

### A matter of `np.float32` and `np.float64`

In [2]:
p = 0.09
side = 128
howmany = 6
seed = 2
npyList = [np.float32, np.float64]

In [3]:
start_letter = 'a'  # Change this to start from a different letter
total_subplots = howmany*2
alphabet = string.ascii_lowercase
start_index = alphabet.index(start_letter.lower())
labels = [fr"\textbf{{({alphabet[(start_index + i) % 26]})}}" for i in range(total_subplots)]

In [4]:
fig, ax = plt.subplots(ncols=howmany, nrows=len(npyList), 
                       figsize=(30, 30*2/howmany), constrained_layout=True)
cmap = ListedColormap(['black', 'red', 'white'])
norm = Normalize(vmin=-1, vmax=1)
#
for cont,npty in enumerate(npyList):
    np.random.seed(seed)
    random.seed(seed)
    #
    l = Lattice2D(side, pflip=p)
    l.flip_random_fract_edges()
    l.compute_k_eigvV(howmany=howmany, MODE_dynspec="scipy", typf=npty)
    #
    axxClabList = zip(ax[cont,:], labels[cont*howmany:(cont+1)*howmany])
    for i, (axx, current_label) in enumerate(axxClabList):
        eius = l.eigV[i].reshape(l.syshape)
        ei = flip_to_positive_majority_adapted(eius)
        result = sign_with_threshold(ei, threshold=np.finfo(npty).eps)
        im = axx.imshow(result, cmap=cmap, interpolation='none')
        axx.set_xticks([])
        axx.set_yticks([])
        if cont == 0:
            axx.set_xlabel(fr"$|\lambda_{i+1}\rangle$")
        axx.text(0.05, 0.9, current_label, transform=axx.transAxes, fontsize=50,
                 fontweight='bold', color='black', ha='left', va='center', 
                 bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', 
                           pad=10))
    minExpRL = np.rint(np.log10(np.finfo(npty).eps)).astype(int)
    cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), 
                        ax=ax[cont, :], extendfrac='auto', pad=0.01,
                        orientation='vertical', extend='both')
    cbar.set_ticks([-1, 0, 1])
    cbar.set_ticklabels(['$-1$', fr'$< |10^{{{minExpRL}}}|$', '$+1$'], 
                        va='center', rotation='vertical')
    cbar.ax.tick_params(axis='y', pad=15)
#
ax[0,0].set_ylabel("float32")
ax[1,0].set_ylabel("float64")
#
fig.savefig(os.path.join(datPthDict['res_limit'], f"howmany{howmany}_comparison_res_limit.pdf"),
            bbox_inches="tight")
plt.close('all')

### A matter of $p$ and $L$

In [5]:
pList = [0.01, 0.07, 0.103, 0.2, 0.5]
LList = [16, 32, 64, 128, 256]
npty = np.float64
seed = 12430184

In [6]:
start_letter = 'm'  # Change this to start from a different letter
total_subplots = 2*max(len(pList), len(LList))
alphabet = string.ascii_lowercase
start_index = alphabet.index(start_letter.lower())
labels = [fr"\textbf{{({alphabet[(start_index + i) % 26]})}}" for i in range(total_subplots)]

In [7]:
cmap = ListedColormap(['black', 'red', 'white'])
def process_subplot(ax, side, p, seed, npty):
    np.random.seed(seed)
    random.seed(seed)
    l = Lattice2D(side, pflip=p)
    l.flip_random_fract_edges()
    l.compute_k_eigvV(howmany=1, MODE_dynspec="scipy", typf=npty)
    eius = l.eigV[0].reshape(l.syshape)
    ei = flip_to_positive_majority_adapted(eius)
    result = sign_with_threshold(ei, threshold=np.finfo(npty).eps)
    ax.imshow(result, cmap=cmap, interpolation='none')
    ax.set_xticks([])
    ax.set_yticks([])

In [8]:
fig, ax = plt.subplots(nrows=2, ncols=max(len(pList), len(LList)), 
                       figsize=(30, 30*2/max(len(pList), len(LList))), constrained_layout=True)
#
L = 128
for i, p in enumerate(pList):
    process_subplot(ax[0, i], L, p, seed, npty)
    ax[0, i].set_xlabel(fr"$p={p:.3g}$")
ax[0, 0].set_ylabel(f"Lattice side: {L}")
#
p = 0.103
for i, L in enumerate(LList):
    process_subplot(ax[1, i], L, p, seed, npty)
    ax[1, i].set_xlabel(f"$L={L}$")
ax[1, 0].set_ylabel(fr"$p={p:.3g}$")
for axx, current_label in zip(flatten(ax), labels):
    axx.text(0.05, 0.9, current_label, transform=axx.transAxes, fontsize=50, 
             fontweight='bold', color='black', ha='left', va='center', 
             bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=10))

norm = Normalize(vmin=-1, vmax=1)  
cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), 
                    ax=ax.ravel().tolist(), extendfrac='auto', pad=0.01,
                    orientation='vertical', extend='both')
cbar.set_ticks([-1, 0, 1])
cbar.set_ticklabels(['$-1$', fr'$< |10^{{{minExpRL}}}|$', '$+1$'], 
                    va='center', rotation='vertical')
cbar.ax.tick_params(axis='y', pad=15) 
fig.savefig(os.path.join(datPthDict['res_limit'], "pN_comparison_res_limit.pdf"), 
            bbox_inches="tight")
plt.close('all')

## Spectral values

### Spectral Distribution

In [ ]:
pflip = 0.103
labelIn = 'e'
l2ddatpath = sgPath2
nAvg = 10000
howmany = 1
minval = 1e-2
mode = 'scipy'
geo = 'squared'
alphabet = string.ascii_lowercase
start_index = alphabet.index(labelIn.lower())
labels = [fr"\textbf{{{alphabet[(start_index + i) % 26]})}}" for i in range(2)]

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10), ncols=2, 
                       gridspec_kw={'width_ratios': [6, 4]})
for side in 2**np.arange(4, 10):
    l = Lattice2D(side, geo, pflip=pflip, sgpath=l2ddatpath)
    kwplot = dict(label=f"{l.N}", zorder=1/side)
    #
    workDir = l.spectpath
    fnameBase = f"dist{howmany}_{pflip:.3g}_{mode}"
    fname = f"{workDir}{fnameBase}_{nAvg}.pkl"
    #
    try:
        with open(fname, "rb") as f:
            listOfCounters = pk.load(f)
    except FileNotFoundError:
        navgdone, _ = os.path.splitext(
            find_matching_files(workDir, fnameBase)[0].split('_')[-1])
        navgdone = int(navgdone)
        fname = f"{workDir}{fnameBase}_{navgdone}.pkl"
        with open(fname, "rb") as f:
            listOfCounters = pk.load(f)
    for ii,i in enumerate(listOfCounters):
        k, v = zip(*sorted(dict(i).items()))
        k = np.array(k)
        #
        smthY = gaussian_filter(v, sigma=1)
        normY = smthY / np.trapz(smthY, k)
        cumY = np.cumsum(normY * np.diff(k, prepend=k[0]))
        #
        posX, negX = k[k > 0], k[k < 0]
        posY, negY = normY[k > 0], normY[k < 0]
        #
        plpl, = ax[0].plot(posX, posY, 'h-', **kwplot)
        ax[0].plot(negX, negY, 'h-', color=plpl.get_color(), **kwplot)
        ax[1].plot(k, cumY, 'h-', **kwplot)
        if np.min(np.abs(k)) < minval: minval = np.min(np.abs(k))

for i, (axx,xpos) in enumerate(zip(ax, [0.03, 0.57])):
    locator = SymmetricalLogLocator(linthresh=minval, base=10)
    axx.set_xscale('symlog', linthresh=minval)
    ticks = np.array([-1, -1e-15, 0, 1e-15, 1])
    axx.set_xticks(ticks)
    axx.xaxis.set_minor_locator(locator)
    axx.set_xlabel(r"$|\lambda_0\rangle_i$")
    axx.set_yscale('log')
    axx.text(xpos, 0.9, labels[i], transform=fig.transFigure, fontsize=50, 
                fontweight='bold', color='black', ha='left', va='center', 
                bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', 
                        pad=10))

ax[0].axvline(np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
ax[0].axvline(-np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
ax[0].set_ylabel(fr"$P(|\lambda_0\rangle)$")
ax[1].set_ylabel(fr"$C(|\lambda_0\rangle)$")
ax[1].axhline(0.5, color='k', linestyle='--', lw=1)
ax[1].legend(fontsize=25)
#
fig.tight_layout()
fig.savefig(os.path.join(datPthDict['res_limit'], f"dist_{side}_{pflip:.3g}_{nAvg}.pdf"),
            bbox_inches='tight', transparent=True)
plt.close('all')

## Other

### Smallest eigenvalue Signed Laplacian

In [ ]:
geo = 'squared'
#
nAvg = 500
bins_count = 500
period = 20
workDir = "data/plot/dev_tool_Lattice2D/"
os.makedirs(workDir, exist_ok=True)
for mode in ['scipy', 'numpy']:
    for side in [16, 32, 64]:
        for pflip in [0.01, 0.05, 0.103, 0.2]:
            #
            fnameBase = f"dist_{side}_{pflip:.3g}_{mode}"
            fname = f"{workDir}{fnameBase}_{nAvg}.pkl"

            if not os.path.exists(fname):
                # Setup and initial calculation
                initial_eig = np.abs(eigV_for_lattice2D(side, pflip=pflip, geo=geo, mode=mode))
                bins, bin_centers = create_symmetric_log_bins(np.min(initial_eig), np.max(initial_eig), bins_count)

                bin_counter = Counter()
                fnameOld = ""
                nAvgDone = 0
                fnameExists = glob.glob(f"{workDir}{fnameBase}*")
                if fnameExists:
                    nAvgDone = int(os.path.splitext(fnameExists[0].split('_')[-1])[0])
                    fnameOld = fnameExists[0]
                nAvgNeed = nAvg - nAvgDone
                for current_period in range((nAvgNeed // period) + bool(nAvgNeed % period)):
                    batch_size = min(nAvgNeed - current_period * period, period)
                    eig_values = np.concatenate([eigV_for_lattice2D(side, pflip=pflip, geo=geo, mode=mode) for _ in range(batch_size)])
                    bin_counter.update(bin_eigenvalues(eig_values, bins, bin_centers))
                    fnameNew = f"{workDir}{fnameBase}_{nAvgDone + (current_period + 1) * period}.pkl"
                    try:
                        os.rename(fnameOld, fnameNew)
                    except FileNotFoundError:
                        pass
                    with open(fnameNew, "wb") as f:
                        pk.dump(dict(bin_counter), f)
                    fnameOld = fnameNew

                # At the end, save the final state if needed
                if nAvgNeed % period:
                    fnameNew = f"{workDir}{fnameBase}_{nAvg}.pkl"
                    os.rename(fnameOld, fnameNew)
                    with open(fnameNew, "wb") as f:
                        pk.dump(dict(bin_counter), f)


In [ ]:

pflip = 0.2

In [ ]:
for side in [16, 32, 64, 96]:
    mode = 'scipy'
    fnameBase = f"dist_{side}_{pflip:.3g}_{mode}"
    fname = f"{workDir}{fnameBase}_{nAvg}.pkl"
    with open(fname, "rb") as f:
        k, v = zip(*sorted(pk.load(f).items()))
    plt.axvline(np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
    plt.axvline(-np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
    plt.plot(k, v, 'o')

    plt.title(fr"$P(|\lambda_0\rangle)$, N={side**2}, {mode}, $p={pflip}$, #{nAvg}")
    plt.xscale('symlog', linthresh=1e-17)
    plt.yscale('log')
    plt.xlabel(r"$|\lambda_0\rangle_i$")
    plt.ylabel(r"$P(|\lambda_0\rangle_i)$")
    # plt.savefig(f"{workDir}dist_{side}_{pflip:.3g}_{nAvg}.png")
    plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
    # k, v = zip(*sorted(pk.load(f).items()))
pflip = 0.5
nAvg = 10000
howmany = 1
mode = 'scipy'
geo = 'squared'
for side in [16, 32, 64, 128, 256, 512]:
# howmany = 1
    l = Lattice2D(side, pflip=pflip, geo=geo, sgpath="cluster_data/spectra_test_1/")
    fnameBase = f"dist{howmany}_{pflip:.3g}_{mode}"
    workDir = l.spectpath
    fname = f"{workDir}{fnameBase}_{nAvg}.pkl"
    try:
        with open(fname, "rb") as f:
            listOfCounters = pk.load(f)
    except FileNotFoundError:
        navgdone, _ = os.path.splitext(find_matching_files(workDir, fnameBase)[0].split('_')[-1])
        navgdone = int(navgdone)
        fname = f"{workDir}{fnameBase}_{navgdone}.pkl"
        with open(fname, "rb") as f:
            listOfCounters = pk.load(f)
    minval = 1e-2
    for ii,i in enumerate(listOfCounters):
        k, v = zip(*sorted(dict(i).items()))
        smthY = gaussian_filter(v, sigma=2)
        area = np.trapz(smthY, k)
        normY = smthY / area
        cumY = np.cumsum(normY * np.diff(k, prepend=k[0]))

        ax[0].plot(k, normY, '-', label=f"{ii:.3g}")
        ax[1].plot(k, cumY, '-', label=f"{ii:.3g}")
        if np.min(np.abs(k)) < minval:
            minval = np.min(np.abs(k))

fig.suptitle(fr"N={side**2}, {mode}, $p={pflip}$, {geo} #{nAvg}")
# ax[0].title(fr"$P(|\lambda_0\rangle)$")
# ax[1].title(fr"$C(|\lambda_0\rangle)$")

for axx in ax:
    locator = SymmetricalLogLocator(linthresh=minval/10, subs=[1.0], base=10)
    axx.set_xscale('symlog', linthresh=minval/10)
    axx.xaxis.set_major_locator(locator)
    labels = axx.get_xticklabels()
    # Set visibility of every second label to False
    for label in labels[::2]:
        if label.get_text() == '$\\mathdefault{0}$':
            continue
        label.set_visible(False)
    axx.set_xlabel(r"$|\lambda_0\rangle_i$")
    # axx.legend(ncol=2)


ax[0].axvline(np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
ax[0].axvline(-np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
ax[0].set_yscale('log')
ax[0].set_ylabel(fr"$P(|\lambda_0\rangle)$")
ax[1].set_ylabel(fr"$C(|\lambda_0\rangle)$")

labels = []
handles = []
for a in ax:
    for handle, label in zip(*a.get_legend_handles_labels()):
        handles.append(handle)
        labels.append(label)

# ax[1].set_yscale('log')
plt.savefig(f"{workDir}dist_{side}_{pflip:.3g}_{nAvg}.png")
# plt.show()

In [ ]:
pflip = 0.2

In [ ]:
for side in [16, 32, 64]:
    mode = 'scipy'
    fnameBase = f"dist_{side}_{pflip:.3g}_{mode}"
    fname = f"{workDir}{fnameBase}_{nAvg}.pkl"
    with open(fname, "rb") as f:
        k, v = zip(*sorted(pk.load(f).items()))
    plt.axvline(np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
    plt.axvline(-np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
    plt.plot(k, v, 'o')
    #
    mode = 'numpy'
    fnameBase = f"dist_{side}_{pflip:.3g}_{mode}"
    fname = f"{workDir}{fnameBase}_{nAvg}.pkl"
    with open(fname, "rb") as f:
        k, v = zip(*sorted(pk.load(f).items()))
    plt.plot(k, v, 'o')

    plt.title(fr"$P(|\lambda_0\rangle)$, N={side**2}, {mode}, $p={pflip}$, #{nAvg}")
    plt.xscale('symlog', linthresh=1e-17)
    plt.yscale('log')
    plt.xlabel(r"$|\lambda_0\rangle_i$")
    plt.ylabel(r"$P(|\lambda_0\rangle_i)$")
    # plt.savefig(f"{workDir}dist_{side}_{pflip:.3g}_{nAvg}.png")
    plt.show()

#### $\min|\lambda_0\rangle$

In [ ]:
mode = 'scipy'
pflip = 0.
for side in [16, 32, 64, 96]:
    fnameBase = f"dist_{side}_{pflip:.3g}_{mode}"
    fname = f"{workDir}{fnameBase}_{nAvg}.pkl"
    with open(fname, "rb") as f:
        k, v = zip(*sorted(pk.load(f).items()))
    plt.plot(side**2, min(np.abs(k)), 'o' , label=f"{side**2}")
plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.savefig(f"{workDir}minl0_{pflip:.3g}_{nAvg}.png")

In [ ]:
%matplotlib inline
#
from lrgsglib.shared import *
from lrgsglib.config.utils import *
from lrgsglib.core import gaussian_filter, Lattice2D, \
    read_files_to_2d_array, move_to_rootf
#
move_to_rootf(True)

In [ ]:
plt.figure()
for p in np.linspace(0.01, 0.5, num=20):
    stdarr = []
    for na in range(100):
        l = Lattice2D(32, pflip=p, geo='squared')
        l.flip_random_fract_edges()
        l.compute_k_eigvV(MODE_dynspec='numpy')
        stdarr.append(np.std(np.diff(l.eigv)))
    plt.scatter(p, np.mean(stdarr), marker='o', c='k')

In [ ]:
p=0.5
l = Lattice2D(128, pflip=p, geo='squared')
l.flip_random_fract_edges()
l.compute_k_eigvV(MODE_dynspec='numpy')


In [ ]:
plt.plot(l.eigv)

In [ ]:
plt.plot(np.diff(l.eigv))

In [ ]:
side = 32
for p in np.linspace(0.01, 0.5):
    l = Lattice2D(side, pflip=p, geo='squared')
    l.flip_random_fract_edges()
    l.compute_k_eigvV(MODE_dynspec='numpy')
    matrix1 = l.sLp.todense().astype(np.float64)
    V1, E1 = l.eigv, l.eigV.T
    np.allclose(matrix1 @ V1, V1 @ np.diag(E1))

In [ ]:

# Setup and initial calculation
initial_eig = np.abs(eigV_for_lattice(side, pflip=pflip, geo=geo, mode=mode))
bins, bin_centers = create_symmetric_log_bins(np.min(initial_eig), np.max(initial_eig), bins_count)

bin_counter = Counter()
fnameOld = ""
if not os.path.exists(fname):
    nAvgDone = 0
    fnameExists = glob.glob(f"{workDir}{fnameBase}*")
    if fnameExists:
        nAvgDone = int(os.path.splitext(fnameExists[0].split('_')[-1])[0])
        fnameOld = fnameExists[0]
    nAvgNeed = nAvg - nAvgDone
    for current_period in range((nAvgNeed // period) + bool(nAvgNeed % period)):
        batch_size = min(nAvgNeed - current_period * period, period)
        eig_values = np.concatenate([eigV_for_lattice(side, pflip=pflip, geo=geo, mode=mode) for _ in range(batch_size)])
        bin_counter.update(bin_eigenvalues(eig_values, bins, bin_centers))
        fnameNew = f"{workDir}{fnameBase}_{nAvgDone + (current_period + 1) * period}.pkl"
        try:
            os.rename(fnameOld, fnameNew)
        except FileNotFoundError:
            pass
        with open(fnameNew, "wb") as f:
            pk.dump(dict(bin_counter), f)
        fnameOld = fnameNew

    # At the end, save the final state if needed
    if nAvgNeed % period:
        fnameNew = f"{workDir}{fnameBase}_{nAvg}.pkl"
        os.rename(fnameOld, fnameNew)
        with open(fnameNew, "wb") as f:
            pk.dump(dict(bin_counter), f)


In [ ]:
mode = 'scipy'
fnameBase = f"dist_{side}_{pflip:.3g}_{mode}"
fname = f"{workDir}{fnameBase}_{nAvg}.pkl"
with open(fname, "rb") as f:
    k, v = zip(*sorted(pk.load(f).items()))
plt.axvline(np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
plt.axvline(-np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
plt.plot(k, v, 'o')
#
mode = 'numpy'
fnameBase = f"dist_{side}_{pflip:.3g}_{mode}"
fname = f"{workDir}{fnameBase}_{nAvg}.pkl"
with open(fname, "rb") as f:
    k, v = zip(*sorted(pk.load(f).items()))
plt.plot(k, v, 'o')

plt.title(fr"$P(|\lambda_0\rangle)$, N={side**2}, {mode}, $p={pflip}$, #{nAvg}")
plt.xscale('symlog', linthresh=1e-17)
plt.yscale('log')
plt.savefig(f"{workDir}dist_{side}_{pflip:.3g}_{nAvg}.png")

In [ ]:
def eigV_for_lattice(side, mode='scipy', **kwargs):
    l = Lattice2D(side, **kwargs)
    l.flip_random_fract_edges()
    l.compute_k_eigvV(MODE_dynspec=mode)
    return flip_to_positive_majority(l.eigV[0])
def create_symmetric_log_bins(min_val, max_val, num_bins, incMagn = 2):
    """Creates symmetric logarithmic bins."""
    bins_positive = np.logspace(np.log10(min_val)-incMagn, np.log10(max_val)+incMagn, num_bins//2)
    bins_negative = -np.flip(bins_positive)
    return np.concatenate((bins_negative, [0], bins_positive))
def bin_eigenvalues(eig_values, bins):
    """Bins eigenvalues and counts occurrences in each bin."""
    indices = np.digitize(eig_values, bins, right=True)
    return Counter(indices)

In [ ]:
# Setup
side = 64
geo = 'squared'
pflip = 0.103
#
nAvg = 200
bins_count = 500
period = 20
#
workDir = "data/plot/dev_tool_Lattice2D/"
mode = 'numpy'
fnameBase = f"dist_{side}_{pflip:.3g}_{mode}"
fname = f"{workDir}{fnameBase}_{nAvg}.pkl"

In [ ]:
# Get initial eigenvalues to define bins
initial_eig = np.abs(eigV_for_lattice(side, pflip=pflip, geo=geo, mode=mode))
print(np.log10(np.max(initial_eig))+4)
bins = create_symmetric_log_bins(np.min(initial_eig), np.max(initial_eig), 
                                 bins_count)
#
bin_counter = Counter()
fnameOld = ""
if not os.path.exists(fname):
    nAvgDone = 0
    fnameExists = glob.glob(f"{workDir}{fnameBase}*")
    if fnameExists:
        nAvgDone = int(os.path.splitext(fnameExists[0].split('_')[-1])[0])
        fnameOld = fnameExists[0]
    nAvgNeed = nAvg - nAvgDone
    for current_period in range((nAvgNeed // period) + bool(nAvgNeed % period)):
        batch_size = min(nAvgNeed - current_period * period, period)
        eig_values = np.concatenate([eigV_for_lattice(side, pflip=pflip, geo=geo, mode=mode) for _ in range(batch_size)])
        bin_counter.update(bin_eigenvalues(eig_values, bins))
        fnameNew = f"{workDir}{fnameBase}_{nAvgDone + (current_period + 1) * period}.pkl"
        try:
            os.rename(fnameOld, fnameNew)
        except FileNotFoundError:
            pass
        with open(fnameNew, "wb") as f:
            pk.dump(dict(bin_counter), f)
        fnameOld = fnameNew

    # At the end, save the final state if needed
    if nAvgNeed % period:
        fnameNew = f"{workDir}{fnameBase}_{nAvg}.pkl"
        os.rename(fnameOld, fnameNew)
        with open(fnameNew, "wb") as f:
            pk.dump(dict(bin_counter), f)

In [ ]:
mode = 'scipy'
fnameBase = f"dist_{side}_{pflip:.3g}_{mode}"
fname = f"{workDir}{fnameBase}_{nAvg}.pkl"
k,v = zip(*pk.load(open(fname, "rb")).items())
binsv = bins[np.array(k)-1]
plt.plot(binsv, v, 'o')
#
mode = 'numpy'
fnameBase = f"dist_{side}_{pflip:.3g}_{mode}"
fname = f"{workDir}{fnameBase}_{nAvg}.pkl"
k,v = zip(*pk.load(open(fname, "rb")).items())
binsv = bins[np.array(k)-1]
plt.plot(binsv, v, 'o')
#
plt.title(fr"$P(|\lambda_0\rangle)$, N={side**2}, {mode}, $p={pflip}$, #{nAvg}")
plt.axvline(np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
plt.axvline(-np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
plt.xscale('symlog', linthresh=1e-17)
plt.yscale('log')
plt.savefig(f"{workDir}dist_{side}_{pflip:.3g}_{nAvg}_{mode}.png")

In [ ]:
l = Lattice2D(128, pflip=0.103, geo='squared')
l.flip_random_fract_edges()
l.compute_k_eigvV(MODE_dynspec='scipy')
np.max(flip_to_positive_majority(l.eigV[0]))

In [ ]:
l = Lattice2D(64, pflip=0.103, geo='squared')
l.flip_random_fract_edges()
l.compute_k_eigvV(MODE_dynspec='scipy')
#
outp, outm = symlog_binning(flip_to_positive_majority(l.eigV[0].flatten()))
plt.plot(outp[0], outp[1], marker='o', lw=1)
plt.plot(outm[0], outm[1], marker='o', lw=1)
plt.xscale('symlog', linthresh=1e-20)
plt.yscale('log')
plt.axvline(np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
plt.axvline(-np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)

### phtra plot

In [ ]:
perc = pd.read_csv("/home/opisthofulax/PhD/projects/complex-networks/LRG-Signed/data/plot/tmp/OrderP_L64",
                       sep=',', header=None)
l = Lattice2D(64, geo='squared', 
            initNwDict=True, 
            with_positions=False,
            sgpath="plot/paper/suppinf/phase_transition_l2d/"
)
data = read_files_to_2d_array(l.phtrapath, 'rand')
data = data[np.argsort(data[:, 0])]
#
pinf2 = gaussian_filter(data[:, 5], sigma=2)
#
chi = gaussian_filter(data[:, 7]*l.N, sigma=1)
#
x_scaled = data[:, 0]
#
chi_scaled = gaussian_filter(perc[3]*64**2,sigma=0)

chiplot, = plt.plot(perc[5], chi_scaled,
                          lw=2, marker='')
chiplot, = plt.plot(x_scaled, chi,
                          lw=2, marker='')

### precision problems

In [ ]:
flatten(ax)

In [ ]:
np.random.seed(0)
random.seed(0)
fig, ax = plt.subplots(ncols=5, figsize=(35,7))


In [ ]:
fig, ax = plt.subplots(ncols=5, nrows=5, figsize=(35,35))
for i in range(10):
    l = Lattice2D(128, pflip=0.12)
    l.flip_random_fract_edges()
    l.compute_k_eigvV(howmany=2)
    ei = l.eigV[0].reshape(l.syshape)
    result = sign_with_threshold(ei, threshold=1e-15)
    ax[i//5][i%5].imshow(result, cmap=cmap, interpolation=None)

In [ ]:
k, v = l.cluster_distribution_list().keys(), l.cluster_distribution_list().values()
plt.plot(k, v, marker='o', lw=0)
plt.yscale('log')
plt.xscale('log')

### Distribution of values first eigenstate

In [ ]:
%matplotlib inline
#
from lrgsglib.shared import *
from lrgsglib.config.utils import *
from lrgsglib.core import gaussian_filter, Lattice2D, \
    read_files_to_2d_array, move_to_rootf
#
move_to_rootf(True)

In [ ]:
l = Lattice2D(64, pflip=0.1, geo='squared')
l.flip_random_fract_edges()
l.compute_k_eigvV(MODE_dynspec='numpy')

In [ ]:
outp, outm = symlog_binning(flip_to_positive_majority(l.eigV[0].flatten()))
plt.plot(outp[0], outp[1], marker='o', lw=1)
plt.plot(outm[0], outm[1], marker='o', lw=1)
plt.xscale('symlog', linthresh=1e-20)
plt.yscale('log')
plt.axvline(np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)
plt.axvline(-np.finfo(np.float64).eps, color='k', linestyle='--', lw=1)

In [ ]:
def flip_edges_optimized(graph, fraction=0.5):
    """
    Flips the sign of a fraction of the +1 edges in the graph to -1 using more concise NetworkX functions.

    Parameters:
    - graph: A networkx graph where edges have a 'weight' attribute.
    - fraction: Fraction of +1 weighted edges to flip. Value between 0 and 1.

    Returns:
    - The modified graph with some edges flipped.
    """
    # Ensure the fraction is between 0 and 1
    if not (0 <= fraction <= 1):
        raise ValueError("Fraction must be between 0 and 1.")
    
    # Find all edges with weight +1
    
    # Calculate the number of edges to flip
    num_to_flip = int(graph.number_of_edges() * fraction)
    
    # Randomly select a subset of positive edges to flip
    edges_to_flip = random.sample(list(graph.edges()), k=num_to_flip)
    
    # Prepare a dictionary with the selected edges and their new weight
    flipped_edges = {graph[i][j]: -1 for (i, j) in edges_to_flip}
    
    # Use set_edge_attributes to apply the new weights
    nx.set_edge_attributes(graph, flipped_edges, 'weight')
    
    return graph

In [ ]:
G = nx.grid_2d_graph(128, 128, periodic=True)
nx.set_edge_attributes(G, +1, 'weight')
flip_edges_optimized(G, 0.5)

In [ ]:
G[(0, 0)][(0, 1)]

In [ ]:


# plt.xlim(-1.5, 1)

In [ ]:
np.where(np.isnan(l.eigV[0].flatten()))

In [ ]:
np.isnan(np.logl.eigV[0].flatten()).any()